In [11]:
using System.Text.RegularExpressions;
using System.IO;
using Newtonsoft.Json;

In [12]:
#nullable enable

struct BVArchive
{
    public string BV; // 视频BV号
    public string Title; // 视频标题
    public string Auther; // 视频作者
    public DateTime PubDate; // 视频发布时间
    public string Description; // 视频简介
    public DateTime ArchiveDate; // 归档时间
    public string? Video; // 是否有视频, 没有就是null, 有就是归档编码
    public string? Danmaku; // 是否有弹幕
    public string? Comment; // 是否有评论

    public BVArchive(string bv, string title, string auther, DateTime pubDate, string description, DateTime archiveDate, string? video, string? danmaku, string? comment)
    {
        BV = bv;
        Title = title;
        Auther = auther;
        PubDate = pubDate;
        Description = description;
        ArchiveDate = archiveDate;
        Video = video;
        Danmaku = danmaku;
        Comment = comment;
    }
}

此处开始是反向编程（预定义后置函数）

In [13]:
BVArchive GetArchiveFromLine(string line)
{
    return JsonConvert.DeserializeObject<BVArchive>(line);
}

In [14]:
const string TRUE_SYMBOL = "⚪";
const string FALSE_SYMBOL = " ";
const string filePath = "Readme.md";
const string JsonPath = "B站视频+评论+弹幕备份.json";
const string md_heading = 
    "# 已归档的视频\n" +
    $"'{TRUE_SYMBOL}' 表示含有, '{FALSE_SYMBOL}' 表示没有\n"+
    "| BV号 | 标题 | 视频 | 弹幕 | 评论 | 作者 |\n" +
    "| :-- | :--: | :--: | :--: | :--: | :-- |\n";

var md_sw = File.CreateText(filePath);
md_sw.Write(md_heading);
md_sw.Flush();

In [15]:
// 这部分是用来 DEBUG 的
// md_sw.Close();
// md_sw.Dispose();

用于删除文本中 非 中文/英文/数字 字符的方法：

In [16]:
string RemoveNonTextCharacters(string input)
{
    return Regex.Replace(input, @"[^\w\u4e00-\u9fa5]", string.Empty);
}

In [17]:
string GetLineStringFromArchive(BVArchive archive)
{
    string bv = archive.BV;
    string title = RemoveNonTextCharacters(archive.Title).Replace("哔哩哔哩","");
    string hasVideo = archive.Video != null ? TRUE_SYMBOL : FALSE_SYMBOL;
    string hasDanmaku = archive.Danmaku != null ? TRUE_SYMBOL : FALSE_SYMBOL;
    string hasComment = archive.Comment != null ? TRUE_SYMBOL : FALSE_SYMBOL;
    string auther = string.Concat(archive.Auther.TakeWhile(x=>x!=' ')); //只保留名字

    return $"| {bv} | {title} | {hasVideo} | {hasDanmaku} | {hasComment} | {auther}";
}

读取 json，输出到 readme

In [18]:
var json_sr = File.OpenText(JsonPath);
while (!json_sr.EndOfStream)
{
    var line = json_sr.ReadLine();
    var archive = GetArchiveFromLine(line);
    var md_line = GetLineStringFromArchive(archive);
    md_sw.WriteLine(md_line);
}

结束后要关闭流哇

In [19]:
md_sw.Flush();
md_sw.Close();
md_sw.Dispose();
json_sr.Close();
json_sr.Dispose();